In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing import image
from PIL import Image
import glob
import random
import cv2
import re

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Pre-configure Plotting Style 
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12)
%config InlineBackend.figure_format = 'retina'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('MovieGenre.csv',encoding="ISO-8859-1")

In [ ]:
train.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [ ]:
train[train['imdbId'].duplicated()]

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
12760,758798,http://www.imdb.com/title/tt758798,Weirdsville (2007),6.5,Comedy|Crime|Drama,https://images-na.ssl-images-amazon.com/images...
12761,486572,http://www.imdb.com/title/tt486572,The Dog Problem (2006),6.3,Comedy,https://images-na.ssl-images-amazon.com/images...
12762,425395,http://www.imdb.com/title/tt425395,Relative Strangers (2006),5.3,Comedy,https://images-na.ssl-images-amazon.com/images...
12763,466664,http://www.imdb.com/title/tt466664,Alien Autopsy (2006),5.8,Biography|Comedy|Sci-Fi,https://images-na.ssl-images-amazon.com/images...
12764,480011,http://www.imdb.com/title/tt480011,Closure (2007),5.6,Thriller,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
13348,460745,http://www.imdb.com/title/tt460745,Chaos Theory (2008),6.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
13349,464907,http://www.imdb.com/title/tt464907,The Boys & Girls Guide to Getting Down (2006),6.6,Comedy,https://images-na.ssl-images-amazon.com/images...
13350,1078188,http://www.imdb.com/title/tt1078188,Boy A (2007),7.7,Drama,https://images-na.ssl-images-amazon.com/images...
13351,116118,http://www.imdb.com/title/tt116118,Doctor Who (1996),6.4,Adventure|Drama|Sci-Fi,https://images-na.ssl-images-amazon.com/images...


In [ ]:
train=train.drop_duplicates(subset='imdbId', keep="first")

In [ ]:
train.shape

(39515, 6)

In [ ]:
train.isnull().sum()

In [ ]:
train=train.dropna()

In [ ]:
train.shape

(38654, 6)

In [ ]:
lst = [os.path.splitext(x)[0] for x in file_list]
lst = list(map(int, lst))
lst=sorted(lst)
print(lst)

[3, 5, 8, 12, 14, 192, 399, 417, 420, 439, 455, 498, 499, 516, 546, 682, 942, 1105, 1223, 1527, 1737, 1789, 2054, 2130, 2381, 2391, 2461, 2544, 2795, 2844, 2985, 3014, 3016, 3037, 3165, 3419, 3471, 3489, 3643, 3740, 3772, 3930, 3952, 4008, 4066, 4101, 4134, 4150, 4181, 4194, 4277, 4288, 4413, 4465, 4635, 4707, 4712, 4873, 4936, 4972, 5044, 5060, 5074, 5078, 5084, 5149, 5198, 5529, 5542, 5557, 5571, 5622, 5810, 5812, 5916, 5960, 6032, 6145, 6177, 6206, 6333, 6414, 6548, 6589, 6668, 6684, 6753, 6780, 6809, 6886, 6895, 7108, 7194, 7338, 7361, 7476, 7558, 7613, 7819, 7832, 8133, 8395, 8443, 8523, 8634, 8663, 8710, 8827, 8874, 8879, 8891, 9062, 9440, 9611, 9652, 9796, 9878, 9893, 9899, 9937, 9976, 10057, 10071, 10155, 10195, 10208, 10247, 10267, 10281, 10307, 10323, 10418, 10726, 10747, 10806, 10821, 11000, 11071, 11130, 11157, 11193, 11237, 11293, 11358, 11362, 11370, 11387, 11508, 11521, 11565, 11652, 11656, 11723, 11833, 11841, 11865, 11870, 11904, 11908, 11909, 11960, 11979, 11984, 1202

In [ ]:
train = train.reset_index(drop=True)

In [ ]:
print(train['Genre'].nunique())

In [ ]:
text_lines = []
with open("new (2).txt") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    text_lines.append(int(stripped_line))

In [ ]:
train=train[~train.imdbId.isin(text_lines)]

In [ ]:
train.shape

(35365, 6)

In [ ]:
L = train['imdbId'].to_list()
L=sorted(L)

In [ ]:
train['newgenre'] = train['Genre'].str.split('|') 

In [ ]:
L = train['newgenre'].to_list()

Llist=list(map(tuple,L))
print("---------------------------------------------------------------------------------------------------------------------------------")
L_array = np.array(Llist,dtype=object)
print(L_array[0])
print("---------------------------------------------------------------------------------------------------------------------------------")

onehot = MultiLabelBinarizer()
onehot_encoded = onehot.fit_transform(L_array)
print(onehot_encoded[0])
print("---------------------------------------------------------------------------------------------------------------------------------")

print(onehot.classes_)

In [ ]:
newdataframe = pd.DataFrame(onehot_encoded,columns=onehot.classes_)
newdataframe['Type']=train['newgenre'].values 
newdataframe['movieid']=train['imdbId'].values

In [ ]:
newdataframe.info()

In [ ]:
newdataframe=newdataframe.sort_values(['movieid'],ascending=True)

In [ ]:
newdataframe.head(12)

In [ ]:
newdataframe = newdataframe.reset_index(drop=True)

In [ ]:
q=newdataframe['movieid'].to_list()
q.sort(reverse=False)
q=list(map(str,q))
print(len(q))
print(q)

In [ ]:
newdataframe.sum()[:-2].sort_values(ascending=False)

In [ ]:
train_images = []

def gen_images():
    for i in tqdm(range(train.shape[0])):
          try:
            img = image.load_img('/content/drive/MyDrive/posters/'+ q[i]+'.jpg')
          except Exception:
              yield img

for x in gen_images():
    train_images.append(x)

In [ ]:
train_images = []

def gen_images():
    for i in tqdm(range(train.shape[0])):
          img = image.load_img('/content/drive/MyDrive/posters/'+ q[i]+'.jpg', target_size=(150, 150, 3))
          img = image.img_to_array(img)
          img = img/255
          yield img

for x in gen_images():
    train_images.append(x)

In [ ]:
Train_data = []

def gen_images():
    for img in tqdm(sorted(glob.glob("/content/drive/MyDrive/posters/*.jpg"),key=lambda x: int(re.findall(r'\d+', x)[0]))):
        x= cv2.imread(img)
        x = cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
        x=cv2.resize(x,dsize=(250,250))
        x=np.array(x , dtype='float32') 
        x /= 255
        yield x

for x in gen_images():
    Train_data.append(x)

In [ ]:
len(Train_data)

In [ ]:
poster_index = list(range(11))
rand_index = random.sample(poster_index, 2)

# Select random posters
poster_1, poster_2 = Train_data[rand_index[0]], Train_data[rand_index[1]]

# Display movie poster and title
fig, ax = plt.subplots(ncols = 4, figsize=(15, 4))
ax[0].imshow(poster_1)
ax[1].imshow(poster_2)
ax[0].set_title('Movie Poster 1', fontsize=14)
ax[1].set_title('Movie Poster 2', fontsize=14)

# Remove axes
for ax in ax:
    ax.axis('off')    
plt.tight_layout()

print('Genre of Movie Poster 1: {} \nGenre of Movie Poster 2: {}'.format(newdataframe['Type'][rand_index[0]],newdataframe['Type'][rand_index[1]]))

In [ ]:
print(rand_index)

In [ ]:
!python images_from_csv_1.py MovieGenre

In [ ]:
!cp -r /content/posters /content/drive/My\Drive

In [ ]:
import os
directory = 'posters'
len(os.listdir(directory))

In [ ]:
L = train['Genre'].to_list()
print(L)
#arraylist = np.asarray(L)

In [ ]:
train.iloc[37651:37653]

In [ ]:
!mkdir posters